In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import openpyxl as xl
from datetime import date
#kitabxanaların import edilməsi

In [ ]:
from psycopg2 import connect
# declare connection instance
conn = connect(
    dbname = "main_db",
    user = "postgres",
    host = "host.docker.internal",
    password = "supersecretpassword"
)

In [2]:
urls=[]
urls.append(req.get("https://international.pte.hu/study-programs/msc-management-and-leadership"))
urls.append(req.get("https://www.kozminski.edu.pl/en/logistics-supply-chain-management-major"))
urls.append(req.get("https://irk.usos.tu.kielce.pl/en-gb/offer/S1_2022-2023_CDZ/programme/B-BUD-EN-ST1/?from=field:B-BUD"))
#urls.append(req.get("https://oferta.byd.pl/computer-control-systems-engineering.665.html"))
urls.append(req.get("https://apply.vistula.edu.pl/en_GB/courses/course/21-tourism-and-recreation"))
#Linklərin oxunması

In [42]:
urls[0]

<body class="wide page-node-302 path-node page-node-type-study-program">
<div id="skip-link">
<a class="visually-hidden focusable" href="#main-content">
        Skip to main content
      </a>
</div>
<div class="dialog-off-canvas-main-canvas" data-off-canvas-main-canvas="">
<div class="dexp-body-inner layout-default preset-default">
<div class="dexp-section color-white" id="section-top-bar" style="background-color:#047889">
<div class="dexp-container">
<div class="row"> <div class="dexp-region col-9 col-lg-7 col-xl region region-top-left">
<div class="block block-block-content block-block-contentb94fae1e-450e-4c10-b938-05f9af456a9a" id="block-evolve-sub-international-gyorslinkek" style="padding-left:10px;;">
<div class="content">
<div class="clearfix text-formatted field field--name-body field--type-text-with-summary field--label-hidden field__item"><div class="top-links">
<div class="top-link"><a href="https://neptun.pte.hu/" target="_blank"><i class="fa fa-graduation-cap"> </i> Neptu

In [3]:
for url_num, url in enumerate(urls):
    soup=bs(url.text)
    body=soup.body
    urls[url_num]=body
#web saytların body hissəsinin seçilməsi

In [161]:
wb = xl.load_workbook("C:/1 Əsas faillar/urls.xlsx")
sheet = wb.active
#Qeyd edilən ünvandakı excelin oxunması

In [162]:
row_=2
today = date.today()
while 1:
    cell = sheet.cell(row = row_, column = 1) 
    if cell.value==None:
        #cell.value = str(today)
        print(cell.value) 
        break
    row_+=1
#Son tarixin seçilməsi

None


In [163]:
def find_change(new_data, url_num):
    #old_list=old_data.split("\n")
    new_list=new_data.split("\n")
    #print(old_list)
    global row_
    global sheet
    for line_num in range(2, row_):
        cell = sheet.cell(row = line_num, column = 3)
        cell_link = sheet.cell(row = line_num, column = 2)
        #print(old)
        find_old=False
        for new in new_list:
            if cell.value==str(new):
                find_old=True
        if find_old==False and cell_link.value=="url "+str(url_num):
            print("Removed line:", line_num,"=", cell.value)
            cell = sheet.cell(row = line_num, column = 5)
            cell.value="Removed"
            cell = sheet.cell(row = line_num, column = 6)
            cell.value=str(today)
    
    for line_num, new in enumerate(new_list):
        find_new=False
        for line_num in range(2, row_):
            cell = sheet.cell(row = line_num, column = 3)
            if cell.value==str(new):
                find_new=True
        
        if find_new==False:
            print("Added line:",line_num,"=", new)
            
            cell = sheet.cell(row = row_, column = 1)
            cell.value=str(today)
            cell = sheet.cell(row = row_, column = 2)
            cell.value="url "+str(url_num)
            cell = sheet.cell(row = row_, column = 3)
            cell.value=str(new)
            cell = sheet.cell(row = row_, column = 4)
            cell.value="no"
            cell = sheet.cell(row = row_, column = 5)
            cell.value="Added"
            row_+=1
                
#Köhnə data ilə təzə datanın fərqinin tapılması

In [164]:
for url_num, url in enumerate(urls, 2):
    url=str(url)
    find_change(url, url_num-1)
#Baş verən dəyişikliyin excele yazılması

Removed line: 10 = <div class="dexp-section color-white" or:#047889">
Removed line: 16 = <div class="top-link"><a "fa fa-graduation-cap"> </i> Neptun</a></div>
Removed line: 19 = <div class="top-lindle</a></div>
Removed line: 21 = <div classn region-top-right">
Added line: 1049 = <div class="dexp-section color-white" id="section-top-bar" style="background-color:#047889">
Added line: 1050 = <div class="top-link"><a href="https://neptun.pte.hu/" target="_blank"><i class="fa fa-graduation-cap"> </i> Neptun</a></div>
Added line: 1051 = <div class="top-link"><a href="https://moodle.pte.hu" target="_blank">Moodle</a></div>
Added line: 1052 = <div class="dexp-region col-auto col-lg col-xl-2 region region-top-right">


In [165]:
wb.save("C:/1 Əsas faillar/urls.xlsx") 
#Excelin yaddaşa yazılması